In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [6]:
active_businesses = pd.read_csv('Datasources/other general/sd_active_businesses_datasd.csv')

In [7]:
active_businesses

,account_key,address_str,apt_suite,bus_start_dt,business_owner_name,business_phone,city,creation_dt,doing_bus_as_name,expiration_dt,lat,lon,naics_code,naics_description,ownership_type,pmb_box,po_box,state,zip,BID
0,2017014278,10640 scripps ranch blvd,101,2017-08-23,miriam chor freitas,(858) 217-5770,san diego,2017-09-27,miriam chor freitas lcsw,2018-08-31,32.911936,-117.110321,62133,offices of other mental health practitioners,sole,NaN,NaN,CA,92131-1095,NaN
1,2017013866,5085 september st,NaN,2015-09-01,robert burke,NaN,san diego,2017-09-27,robert burke,2018-08-31,32.780080,-117.187049,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92110-1222,NaN
2,2017013873,827 beryl st,NaN,2015-09-01,robert doll,NaN,san diego,2017-09-27,robert doll,2018-08-31,32.803056,-117.256734,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92109-2004,NaN
3,2017013871,2704 boston ave,NaN,2015-09-01,jose m espinozanaranjo & maria m dadaespinoza,NaN,san diego,2017-09-27,jose m espinozanaranjo & maria m dadaespinoza,2018-08-31,32.694014,-117.135811,541,"professional, scientific & technical services",h-w,NaN,NaN,CA,92113-3708,NaN
4,2017013870,1502 mesa brook st,NaN,2015-09-01,feliz h blanco & idalia contreras ochoa,NaN,san diego,2017-09-27,feliz h blanco & idalia contreras ochoa,2018-08-31,32.693291,-117.048771,541,"professional, scientific & technical services",h-w,NaN,NaN,CA,92114-7875,NaN
5,2017013869,4775 monongahela st,NaN,2015-09-01,paul v saadieh,NaN,san diego,2017-09-27,paul v saadieh,2018-08-31,32.834477,-117.228387,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92117-2419,NaN
6,2017013867,302 washington st,150-31,2015-09-01,scott corich,NaN,san diego,2017-09-27,scott corich,2018-08-31,32.750293,-117.162618,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92103-2110,Hillcrest
7,2017013865,2464 fenton pkwy,207,2015-09-01,eduardo d galvan & nallely p padilla,NaN,san diego,2017-09-27,eduardo d galvan & nallely p padilla,2018-08-31,32.780464,-117.131484,541,"professional, scientific & technical services",h-w,NaN,NaN,CA,92108-6705,NaN
8,2017013874,2616 highland ave,5,2015-09-01,ketkeo inthavong,NaN,san diego,2017-09-27,ketkeo inthavong,2018-08-31,32.733630,-117.098926,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92105-4371,NaN
9,2017013863,4584 f st,NaN,2015-09-01,clarence e davis sr,NaN,san diego,2017-09-27,clarence e davis sr,2018-08-31,32.713745,-117.095833,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92102-3664,NaN


In [8]:
active_businesses[['lon', 'lat']]

,lon,lat
0,-117.110321,32.911936
1,-117.187049,32.780080
2,-117.256734,32.803056
3,-117.135811,32.694014
4,-117.048771,32.693291
5,-117.228387,32.834477
6,-117.162618,32.750293
7,-117.131484,32.780464
8,-117.098926,32.733630
9,-117.095833,32.713745


In [9]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [10]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [11]:
schema_two = StructType([StructField("account_key", IntegerType(), True),
                    StructField("lon", DoubleType(), True),
                    StructField("lat", DoubleType(), True)])

In [12]:
spql_active_businesses_info = sqlContext.createDataFrame(active_businesses[['account_key', 'lon', 'lat']], schema_two)

In [13]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [14]:
spql_active_businesses_info_rdd = spql_active_businesses_info.rdd.map(lambda row:(row.account_key, row.lon,row.lat))

In [15]:
spql_listings_and_businesses_info_rdd = spql_listings_info_rdd.cartesian(spql_active_businesses_info_rdd)

In [16]:
spql_listings_and_businesses_info_rdd = spql_listings_and_businesses_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [17]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [18]:
spql_listings_and_businesses_info_rdd = spql_listings_and_businesses_info_rdd.map(distance_calculation)

In [19]:
spql_listings_and_businesses_info_rdd.take(2)

[(11204286,
  -117.00194036983295,
  32.581881678016465,
  2017014278,
  -117.11032140000002,
  32.9119358,
  19.0371639407603),
 (11204286,
  -117.00194036983295,
  32.581881678016465,
  2017013866,
  -117.187049,
  32.78008,
  14.016453587380838)]

In [20]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("account_key", IntegerType(), True),
                    StructField("business_lon", DoubleType(), True),
                    StructField("business_lat", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [21]:
spql_listings_and_businesses_info_df = sqlContext.createDataFrame(spql_listings_and_businesses_info_rdd, schema_three)

In [22]:
spql_listings_and_businesses_info_rdd = spql_listings_and_businesses_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.account_key, row.business_lon, row.business_lat, row.distance))

In [23]:
spql_listings_and_businesses_info_rdd.map(lambda x: x[-1]).take(10)

[19.0371639407603,
 14.016453587380838,
 17.127117708310156,
 8.840158625976906,
 6.5706720082094145,
 17.59107540499231,
 12.009349782157923,
 12.595430537700668,
 9.580656248019492,
 8.547930443182247]

In [24]:
close_businesses_count_one = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_businesses_count_three = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_businesses_count_five = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_businesses_count_ten = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [25]:
close_businesses_count = spql_listings_and_businesses_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [26]:
closest_business_distance = spql_listings_and_businesses_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [27]:
count = 1
Writer = pd.ExcelWriter('listings_business_information.xlsx')

for w in [close_businesses_count_one, close_businesses_count_three, close_businesses_count_five, close_businesses_count_ten, close_businesses_count, closest_business_distance]:
    listings_businesses_information = w.toDF().toPandas()
    listings_businesses_information.to_excel(Writer, 'Sheet' + str(count))
    count = count + 1
    
Writer.save()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:152)
	at org.apache.spark.InterruptibleIterator.next(InterruptibleIterator.scala:43)
	at scala.collection.

In [29]:
business_distance  = closest_business_distance.map(lambda x: x[1])

In [31]:
business_distance.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 9, localhost, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:152)
	at org.apache.spark.InterruptibleIterator.next(InterruptibleIterator.scala:43)
	at scala.collection.